### 爬虫Download 函数

In [4]:
import urllib2

def download(url,user_agent='wswp',num_retries=3):
    print 'Downloading:',url
    headers={'User-agent':user_agent}
    request=urllib2.Request(url,headers=headers)
    try:
        html=urllib2.urlopen(url).read()
    except urllib2.URLError as e:
        print 'Download error:',e,'==>',e.reason
        html=None
        if num_retries>0:
            if hasattr(e,'code') and 500<=e.code<600:
                return download(url,user_agent,num_retries-2)
    return html

url='http://www.vision.uestc.edu.cn'
url1='http://www.metup.com/'
url2='http://httpstat.us/500'
url3='http://blog.tianya.cn/blog-773339-1.shtml'
download(url2)

Downloading: http://httpstat.us/500
Download error: HTTP Error 500: Internal Server Error ==> Internal Server Error
Downloading: http://httpstat.us/500
Download error: HTTP Error 500: Internal Server Error ==> Internal Server Error
Downloading: http://httpstat.us/500
Download error: HTTP Error 500: Internal Server Error ==> Internal Server Error


### 1.网站地图爬取：

In [8]:
import re
def crawl_sitemap(url):
    #download the sitemap file
    sitemap=download(url)
    #extract the sitemap links
    links=re.findall('<loc>(.*?)</loc>',sitemap)   #正则表达式
    #find all links:
    for link in links:
        print link
#         html=download(link)

In [9]:
url='https://jandan.net/sitemaps/2017-09.xml'
crawl_sitemap(url)

Downloading: https://jandan.net/sitemaps/2017-09.xml
http://jandan.net/2017/09/30/historical-fact-2.html
http://jandan.net/2017/09/30/slow-walkers.html
http://jandan.net/2017/09/30/sterile-mosquitoes.html
http://jandan.net/2017/09/30/worst-shopping.html
http://jandan.net/2017/09/30/avoid-bringi.html
http://jandan.net/2017/09/30/next-rocket.html
http://jandan.net/2017/09/30/cold-weather-2.html
http://jandan.net/2017/09/30/eaten-alive.html
http://jandan.net/2017/09/29/brain-appetite.html
http://jandan.net/2017/09/29/pizza-falsch.html
http://jandan.net/2017/09/29/tiny-robots.html
http://jandan.net/2017/09/29/carbon-nanotube.html
http://jandan.net/2017/09/29/human-embryo-2.html
http://jandan.net/2017/09/29/new-giant-rat.html
http://jandan.net/2017/09/29/egyptian-kings.html
http://jandan.net/2017/09/29/viking-boat.html
http://jandan.net/2017/09/29/interesting-facts-behavior.html
http://jandan.net/2017/09/29/cold-weather.html
http://jandan.net/2017/09/29/fish-individual.html
http://jandan.ne

### 2.ID遍历爬虫

#### python 延迟几秒
import time  
time.sleep(3)

In [10]:
import itertools,time
max_errors=5
num_errors=0
for page in range(10):#itertools.count(1):
    url='http://example.webscraping.com/places/default/view/%d' %page
    html=download(url)
    time.sleep(2)
    if html==None:
        num_errors+=1
        if num_errors==max_errors:
            break
    else:
        num_errors=0
        print url

Downloading: http://example.webscraping.com/places/default/view/0
Download error: HTTP Error 404: NOT FOUND ==> NOT FOUND
Downloading: http://example.webscraping.com/places/default/view/1
http://example.webscraping.com/places/default/view/1
Downloading: http://example.webscraping.com/places/default/view/2
http://example.webscraping.com/places/default/view/2
Downloading: http://example.webscraping.com/places/default/view/3
http://example.webscraping.com/places/default/view/3
Downloading: http://example.webscraping.com/places/default/view/4
http://example.webscraping.com/places/default/view/4
Downloading: http://example.webscraping.com/places/default/view/5
http://example.webscraping.com/places/default/view/5
Downloading: http://example.webscraping.com/places/default/view/6
http://example.webscraping.com/places/default/view/6
Downloading: http://example.webscraping.com/places/default/view/7
http://example.webscraping.com/places/default/view/7
Downloading: http://example.webscraping.com/p

### 3.链接爬虫

In [13]:
import re
import urlparse
def link_crawler(seed_url, link_regex):
    """Crawl from the given seed URL following links matched by link_regex"""
    crawl_queue=[seed_url]
    seen=set(crawl_queue)
    while crawl_queue:
        url=crawl_queue.pop()
        print 'link_crawler:',url
        html=download(url)   #暂时不下载
        # 使用link_regex过滤给出的链接
        for link in get_links(html):
            #if re.match(link_regex,link):        ==书上使用match有误，应该为search() 
            if re.search(link_regex,link): 
                link=urlparse.urljoin(seed_url,link)
                if link not in seen:
                    seen.add(link)
                    crawl_queue.append(link)

def get_links(html):
    webpage_regex=re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
    return webpage_regex.findall(html)

# 测试
url='http://example.webscraping.com'
#regex=''
link_crawler('http://example.webscraping.com','/(index|view)')

TypeError: 'type' object has no attribute '__getitem__'

#### 解析robots.txt
使用Python自带的**robotparser模块**

In [14]:
import robotparser
rp=robotparser.RobotFileParser()
rp.set_url('https://www.douban.com/robots.txt')
rp.read()
url='https://www.douban.com/'
user_agent='Wandoujia Spider'
rp.can_fetch(user_agent,url)

False

In [15]:
user_agent='BadCrawler'
rp.can_fetch(user_agent,url)

True

## 数据抓取的三种方法

#### 1.正则表达式

In [19]:
import re
url='http://example.webscraping.com/places/default/view/Afghanistan-1'
html=download(url)
re.findall('<td class="w2p_fw">(.*?)</td>',html)[1]

Downloading: http://example.webscraping.com/places/default/view/Afghanistan-1


'647,500 square kilometres'

#### 2.BeatifulSoup

In [28]:
# html 解析 并 格式规范化
from bs4 import BeautifulSoup
broken_html='<ul class=country><li>Area</li><li>Population</ul>'
soup=BeautifulSoup(broken_html,'html.parser')
fixed_html=soup.prettify()
print fixed_html

<ul class="country">
 <li>
  Area
 </li>
 <li>
  Population
 </li>
</ul>


In [31]:
ul=soup.find('ul',attrs={'class':'country'})
print ul

<ul class="country"><li>Area</li><li>Population</li></ul>


In [32]:
ul.find('li')         # 只查找第一个

<li>Area</li>

In [55]:
all=ul.find_all('li')     #查找所有的项
print all

[<li>Area</li>, <li>Population</li>]


In [56]:
print all[0].text
print all[1].text

Area
Population


In [34]:
print ul.find('li')

<li>Area</li>


##### 使用案例

In [43]:
from bs4 import BeautifulSoup
url='http://example.webscraping.com/places/default/view/239'
html=download(url)
soup=BeautifulSoup(html,'html')
tr=soup.find(attrs={'id':'places_area__row'})
td=tr.find(attrs={'class':'w2p_fw'})
print td.text

Downloading: http://example.webscraping.com/places/default/view/239
244,820 square kilometres


#### 3. lxml

In [64]:
import lxml.html
broken_html='<ul class=country><li>Area<li>Population</ul>'
tree=lxml.html.fromstring(broken_html)
fixed_html=lxml.html.tostring(tree,pretty_print=True)
print fixed_html
type(tree)

<ul class="country">
<li>Area</li>
<li>Population</li>
</ul>



lxml.html.HtmlElement

In [75]:
tree=lxml.html.fromstring(html)
td=tree.cssselect('tr#places_area__row > td.w2p_fw')[0]
area=td.text_content()
print area

244,820 square kilometres


#### 另外的测试

In [86]:
xxx=download
url='https://jandan.net/pic/'

import time
print time.time()

1508595375.78


In [83]:
url='https://jandan.net/pic/'
html=download(url)
soup=BeautifulSoup(html,'html')
soup.find_all('img')

Downloading: https://jandan.net/pic/


[<img onload="add_img_loading_mask(this, load_sina_gif);" org_src="//wx1.sinaimg.cn/mw690/a8e6d38dly1fkq6305vvfg20dc0dcnpj.gif" src="//wx1.sinaimg.cn/thumb180/a8e6d38dly1fkq6305vvfg20dc0dcnpj.gif"/>,
 <img src="//wx3.sinaimg.cn/mw600/6c560b83ly1fkq5r6zhnej20ku0ku3yv.jpg"/>,
 <img src="//wx2.sinaimg.cn/mw600/6c560b83ly1fkq5r7n89wj20ku0u5ta8.jpg"/>,
 <img src="//wx2.sinaimg.cn/mw600/6c560b83ly1fkq5r80jvzj20ku0eewfm.jpg"/>,
 <img src="//wx4.sinaimg.cn/mw600/6c560b83ly1fkq5r88ht5j20ku0ee750.jpg"/>,
 <img src="//wx4.sinaimg.cn/mw600/71021e17gy1fkpb58xbldj20j60dx3zm.jpg"/>,
 <img src="//wx4.sinaimg.cn/mw600/71021e17gy1fkpb59av8fj20hs0dc3zh.jpg"/>,
 <img src="//wx2.sinaimg.cn/mw600/71021e17gy1fkpb59oy7nj20go0ciq3r.jpg"/>,
 <img src="//wx3.sinaimg.cn/mw600/71021e17gy1fkpb58i6nfj20go0cidgu.jpg"/>,
 <img src="//wx3.sinaimg.cn/mw600/71021e17gy1fkpb5aepibj20ep0hsq46.jpg"/>,
 <img src="//wx2.sinaimg.cn/mw600/71021e17gy1fkpb5b0kgwj20go0m80u6.jpg"/>,
 <img src="//wx3.sinaimg.cn/mw600/71021e17gy1fkpb5

In [90]:
import csv
file=open('countries.csv','w')
writer=csv.writer(file)
fields=('area','pupulation','iso','country','captial')
writer.writerow(fields)
file.close()